In [1]:
import pandas as pd
import os 
from sklearn.feature_selection import SelectFromModel
# from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier  

data_path = '/legodata/zhaoj/cvd_risk_time2/data_1_8/processed'
# def get_cohort(data_path, fram, cmd_arg):
#     if fram:
#         if cmd_arg.impute_0:
#             df_cohort = pd.read_csv(os.path.join(data_path, 'df_fram_cohort_0.csv'))
#         else:
#             df_cohort = pd.read_csv(os.path.join(data_path, 'df_fram_cohort.csv'))
#     else:
#         if cmd_arg.impute_0:
#             df_cohort = pd.read_csv(os.path.join(data_path, 'df_full_cohort_0.csv'))
#             df_cohort = df_cohort.drop(['predict'], axis=1)
#         else:
#             df_cohort = pd.read_csv(os.path.join(data_path, 'df_full_cohort.csv'))
#             df_cohort = df_cohort.drop(['predict'], axis=1)

#     if cmd_arg.phemed_bin:
#         df_cohort = convert_phemed_to_bin(df_cohort)

#     if not cmd_arg.demo:
#         print('drop_demo')
#         df_cohort = df_cohort.drop(['AGE_AT_BASELINE', 'RACE_W', 'RACE_B', 'RACE_U', 'GENDER'], axis=1)

#     # if cmd_arg.norm:
#     #     df_cohort = norm_feat(df_cohort)

#     if cmd_arg.reg:
#         # df_cohort = l1_regularization(df=df_cohort,
#         #                               alpha_inv=100)
#         df_cohort = xgb_regularization(df_cohort=df_cohort)

#     return df_cohort

def convert_phemed_to_bin(df_cohort):
    print('CONVERTING PHEMED TO BIN')
    col_lst = list(df_cohort.columns)

    med_col_lst = [col for col in col_lst if 'med_' in col]
    phe_col_lst = [col for col in col_lst if 'phe_' in col]

    df_med = df_cohort[med_col_lst]
    df_phe = df_cohort[phe_col_lst]

    for col in med_col_lst:
        df_med[col] = df_med[col].map(lambda x: 'yes' if x != 0 else 'no')
    df_cohort[med_col_lst] = pd.get_dummies(df_med, drop_first=True)

    for col in phe_col_lst:
        df_phe[col] = df_phe[col].map(lambda x: 'yes' if x != 0 else 'no')
    df_cohort[phe_col_lst] = pd.get_dummies(df_phe, drop_first=True)

    return df_cohort


def create_matched_kfold_splits(df_map, df_cohort, num_splits):
    df_case = df_cohort.sample(frac=1)
    df_case = df_case.loc[df_case['CLASS'] == 1]

    num_case_per_split = len(df_case) // num_splits

    kfold_dict = {}

    for i in range(num_splits):
        fold_dict = {}

        if i == (num_splits - 1):
            df_case_split = df_case.iloc[i * num_case_per_split:]
        else:
            df_case_split = df_case.iloc[i * num_case_per_split:(i+1) * num_case_per_split]

        print('split: {}\t # of cases: {}'.format(i, len(df_case_split)))

        df_matched_controls = df_case_split.merge(df_map,
                                                  how='inner',
                                                  on='GRID')

        print('\t\t # of matched controls: {}'.format(len(df_matched_controls)))

        test_case_control_lst = list(df_case_split['GRID'])

        test_case_control_lst.extend(list(df_matched_controls['GRID_CONTROL']))

        fold_dict['test'] = test_case_control_lst

        print('\t\t # of unique cases and matched controls:\t{}'.format(len(set(test_case_control_lst))))

        train_case_control_lst = list(set(df_cohort['GRID']) - set(test_case_control_lst))

        fold_dict['train'] = train_case_control_lst

        kfold_dict[i] = fold_dict

    return kfold_dict


def std_scale_train_test(df_train, df_test):
    print('BEGIN STD SCALE')

    X_train = df_train.drop(['CLASS', 'GRID'], axis=1)
    X_test = df_test.drop(['CLASS', 'GRID'], axis=1)

    std_scaler = StandardScaler()
    std_scaler.fit(X_train)

    X_train_std = std_scaler.transform(X_train)
    X_test_std = std_scaler.transform(X_test)

    X_train_std = pd.DataFrame(X_train_std)
    X_train_std.columns = list(X_train.columns)

    X_test_std = pd.DataFrame(X_test_std)
    X_test_std.columns = list(X_test.columns)

    X_test_std['CLASS'] = list(df_test['CLASS'])
    X_test_std['GRID'] = list(df_test['GRID'])

    X_train_std['CLASS'] = list(df_train['CLASS'])
    X_train_std['GRID'] = list(df_train['GRID'])

    return X_train_std, X_test_std


def xgb_regularization(df_cohort, thresh):
    df_features = df_cohort.drop(['GRID', 'CLASS'], axis=1)

    clf_feat = SelectFromModel(estimator=XGBClassifier().fit(X=df_features,
                                                             y=df_cohort['CLASS']),
                               prefit=True, threshold=thresh)

    clf_feat.transform(df_features)

    feat_mask = list(clf_feat.get_support())
    lst_select_feat = list()
    for it in range(len(feat_mask)):
        if feat_mask[it]:
            lst_select_feat.append(list(df_features.columns)[it])

    df_cohort_select_feat = df_cohort[['GRID', 'CLASS'] + lst_select_feat]

    print('NUM FEATURES SELECTED: {}'.format(len(df_cohort_select_feat.columns)))

    return df_cohort_select_feat


# lab_lst = [col for col in list(df_cohort.columns) if '(' in col]

In [2]:
# if cmd_arg.impute_0:
#     df_cohort = pd.read_csv(os.path.join(data_path, 'df_fram_cohort_0.csv'))
# else:
#     df_cohort = pd.read_csv(os.path.join(data_path, 'df_fram_cohort.csv'))
# if cmd_arg.impute_0:
#     df_cohort = pd.read_csv(os.path.join(data_path, 'df_full_cohort_0.csv'))
#     df_cohort = df_cohort.drop(['predict'], axis=1)
# else:
#     df_cohort = pd.read_csv(os.path.join(data_path, 'df_full_cohort.csv'))
#     df_cohort = df_cohort.drop(['predict'], axis=1)

df_fram_cohort = pd.read_csv(os.path.join(data_path, 'df_fram_cohort.csv'))

# df_fram_cohort = convert_phemed_to_bin(df_fram_cohort)

df_fram_cohort = df_fram_cohort.drop(['AGE_AT_BASELINE', 'RACE_W', 'RACE_B', 'RACE_U', 'GENDER'], axis=1)

    # df_cohort = l1_regularization(df=df_cohort,
    #                               alpha_inv=100)

In [3]:
# df_fram_cohort_reg = xgb_regularization(df_cohort=df_fram_cohort, thresh='10*mean')
# df_fram_cohort_reg.columns

In [38]:
def run_xgboost(df_train, df_test, params, fold):
    clf = GradientBoostingClassifier(**params)

    X_train, y_train = df_train.drop(['GRID', 'CLASS'], axis=1), df_train['CLASS']
    X_test, y_test = df_test.drop(['GRID', 'CLASS'], axis=1), df_test['CLASS']

    print(clf.get_params())

    # print(X_train.head())

    clf.fit(X=X_train,
            y=y_train,
#             eval_set=[(X_train, y_train), (X_test, y_test)],
#             eval_metric='logloss',
            early_stopping_rounds=8,
            verbose=True)

#     eval_results = clf.evals_result()
    df_eval = pd.DataFrame({**(list(eval_results.values())[0]), **(list(eval_results.values())[1])})

    df_pred = get_pred(clf=clf,
                            df_test=df_test,
                            fold=fold,
                       model='xgb')

    df_feat = pd.DataFrame({'FEATURE': list(X_train.columns), 'WEIGHT': list(clf.feature_importances_)})

    return {'pred': df_pred, 'feat': df_feat, 'eval': df_eval}


# def run_logistic_regression(df_test, df_train, params, fold):
#     print('Run logistic regression')

#     X_train, y_train = df_train.drop(['GRID', 'CLASS'], axis=1), df_train['CLASS']
#     X_test, y_test = df_test.drop(['GRID', 'CLASS'], axis=1), df_test['CLASS']

#     # print(X_train.head())

#     clf = LogisticRegression(**params)

#     print(clf.get_params())

#     ts1 = time.time()
#     clf.fit(X=X_train,
#             y=y_train)
#     ts2 = time.time()
#     print('computation time:\t{}'.format(ts2-ts1))

#     df_pred = get_pred(clf=clf,
#                        df_test=df_test,
#                        fold=fold,
#                        model='lr')

#     df_feat = pd.DataFrame({'FEATURE': list(X_train.columns), 'WEIGHT': clf.coef_.T.flatten()})

#     return {'pred': df_pred, 'feat': df_feat}


# def run_random_forest(df_train, df_test, params, fold):
#     print('run random forest')
#     X_train, y_train = df_train.drop(['GRID', 'CLASS'], axis=1), df_train['CLASS']
#     X_test, y_test = df_test.drop(['GRID', 'CLASS'], axis=1), df_test['CLASS']

#     # print(X_train.head())

#     clf = RandomForestClassifier(**params)

#     print(clf.get_params)

#     ts1 = time.time()
#     clf.fit(X=X_train,
#             y=y_train)
#     ts2 = time.time()
#     print('computation time:\t{}'.format(ts2-ts1))

#     df_pred = get_pred(clf=clf,
#                        df_test=df_test,
#                        fold=fold,
#                        model='rf')

#     # print(clf.feature_importances_.shape)
#     df_feat = pd.DataFrame({'FEATURE': list(X_train.columns), 'WEIGHT': clf.feature_importances_})

#     return {'pred': df_pred, 'feat': df_feat}

In [20]:
def get_params():
    params_lr = {'class_weight': 'balanced',
                 'solver': 'sag',
                 'verbose': 1, 'random_state': 1,
                 'n_jobs': 1,
                 'C': 1}

    params_rf = {'n_estimators': 400,
                 'max_features': .1,
                 'max_depth': 8,
                 'bootstrap': True,
                 'verbose': 1,
                 'class_weight': 'balanced',
                 'min_samples_leaf': 1,
                 'random_state': 1}

    params_xgb = {'max_depth': 8, 'learning_rate': 0.1,
                  'n_estimators': 400, 'silent': False,
                  'objective': 'binary:logistic',
                  'booster': 'gbtree', 'n_jobs': 4,
                  'subsample': 1.0, 'scale_pos_weight': 10, 'random_state': 1}
    
    params_gbt = {'verbose': 1, 'max_depth': 8}

#     if not cmd_arg.cw:
#         params_lr.pop('class_weight', 0)
#         params_rf.pop('class_weight', 0)
#         params_xgb.pop('scale_pos_weight', 0)

    model_params = {'lr': params_lr, 'rf': params_rf, 'xgb': params_xgb, 'gbt': params_gbt}

    return model_params

In [21]:
def get_pred(clf, df_test, fold, model):
    X_test = df_test.drop(['GRID', 'CLASS'], axis=1)
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)

    df_pred = pd.DataFrame({'GRID': list(df_test['GRID']),
                            'PRED': list(y_pred)})
    df_pred['PROB(0)'] = y_prob[:, 0]
    df_pred['PROB(1)'] = y_prob[:, 1]

    df_pred['CLASS'] = list(df_test['CLASS'])

    if model == 'lr' or 'gbt':
        df_pred['SCORE'] = clf.decision_function(X_test)
    else:
        df_pred['SCORE'] = df_pred['PROB(1)']

    df_pred['FOLD'] = [fold] * len(df_pred)

    return df_pred

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

n_folds=10

df_cohort = df_fram_cohort

skf = StratifiedKFold(n_splits=n_folds)
skf.get_n_splits(X=df_cohort,
                 y=df_cohort['CLASS'])

feat_lst = list(df_cohort.columns)
feat_lst.remove('CLASS')
feat_lst.remove('GRID')

# df_pred_cv = pd.DataFrame()
# df_feat_cv = pd.DataFrame({'FEATURE': feat_lst})

print('# of subjects in cohort:\t{}'.format(len(df_cohort)))
print('# of features in cohort:\t{}'.format(len(df_cohort.columns)))

# fold = 0
# for train_index, test_index in skf.split(X=df_cohort,
#                                          y=df_cohort['CLASS']):
kfold_dict = create_matched_kfold_splits(df_map=pd.read_csv(os.path.join('/legodata/zhaoj/cvd_risk_time2/data_1_8', 'raw', 'control_case_mappings.csv')),
                                         df_cohort=df_cohort,
                                         num_splits=n_folds)

fold = 0 

params = get_params()

for fold in range(n_folds):
    train_index = (kfold_dict[fold])['train']
    test_index = (kfold_dict[fold])['test']

    print('FOLD: {}'.format(fold))

    # df_train, df_test = df_cohort.iloc[train_index], df_cohort.iloc[test_index]
    df_train, df_test = df_cohort.loc[df_cohort['GRID'].isin(train_index)], df_cohort.loc[df_cohort['GRID'].isin(test_index)]

    print('\t# of train subject: {}'.format(len(df_train)))
    print('\t# of test subject: {}'.format(len(df_test)))

    df_train, df_test = std_scale_train_test(df_train=df_train,
                                             df_test=df_test)

#         clf_dict = run_logistic_regression(df_train=df_train,
#                                            df_test=df_test,
#                                            params=params['lr'],
#                                            fold=fold)
#         clf_dict = run_random_forest(df_train=df_train,
#                                      df_test=df_test,
#                                      params=params['rf'],
#                                      fold=fold)

    clf = GradientBoostingClassifier(**(params['gbt']))
    

    X_train, y_train = df_train.drop(['GRID', 'CLASS'], axis=1), df_train['CLASS']
    X_test, y_test = df_test.drop(['GRID', 'CLASS'], axis=1), df_test['CLASS']

    print(clf.get_params())

    # print(X_train.head())

    clf.fit(X=X_train,
            y=y_train) 

#     eval_results = clf.evals_result()
#     df_eval = pd.DataFrame({**(list(eval_results.values())[0]), **(list(eval_results.values())[1])})

    df_pred = get_pred(clf=clf,
                            df_test=df_test,
                            fold=fold,
                       model='gbt')

#     df_feat = pd.DataFrame({'FEATURE': list(X_train.columns), 'WEIGHT': list(clf.feature_importances_)})

    # if model == 'rf':
    #     print(clf_dict['pred'])

#     df_pred_cv = pd.concat([df_pred_cv, clf_dict['pred']])
#     df_feat_rank = clf_dict['feat']
#     df_feat_rank = df_feat_rank.rename(columns={'WEIGHT': 'WEIGHT{}'.format(fold)})
    # needs to be inner so that feature reduction will allow # of features in df_feat_cv to be reduced
#     df_feat_cv = df_feat_cv.merge(df_feat_rank, on='FEATURE', how='inner')

    # print(df_feat_rank.sort_values(by='WEIGHT{}'.format(fold), ascending=False).head())
    break 

    fold += 1

# of subjects in cohort:	93348
# of features in cohort:	2634
split: 0	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 1	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 2	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 3	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 4	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 5	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 6	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 7	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched controls:	9333
split: 8	 # of cases: 1037
		 # of matched controls: 8296
		 # of unique cases and matched 

In [ ]:
df_feat = pd.DataFrame({'feature': feat_lst, 'weight': list(clf.feature_importances_)})
df_feat.sort_values(by='weight', ascending=False).head(20)